In [144]:
#####Group Lassoによるマルチタスク学習#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [154]:
####データの生成####
##データの設定
g = 15   #グループ数
T = 50   #タスク数
n1 = np.random.poisson(np.random.gamma(7.5, 1/0.005, T))   
n2 = np.random.poisson(np.random.gamma(5.0, 1/0.005, T))   
N1 = np.sum(n1)
N2 = np.sum(n2)

In [155]:
##入力変数と応答変数を生成
#トピック分布の設定
k = 1000
topic = 30
theta = np.random.dirichlet(np.repeat(0.2, topic), T)
phi = np.random.dirichlet(np.repeat(0.2, k), topic)

#パラメータを生成
beta = np.vstack((np.random.normal(10.0, 1.5, T), np.random.multivariate_normal(np.repeat(0, T), np.diag(np.repeat(0.5, T)), k)))
beta[1:, ] = beta[1:, ] * np.random.binomial(1, 0.4, k)[:, np.newaxis]
beta[1:, ] = beta[1:, ] * np.random.binomial(1, 0.25, k*T).reshape(k, T)
Sigma = np.sqrt(1/np.random.gamma(10/2, 1/10, T))
betat = beta.copy(); Sigmat = Sigma.copy()

#タスクごとにデータを生成
x1_list = [i for i in range(T)]
x2_list = [i for i in range(T)]
y1_list = [i for i in range(T)]
y2_list = [i for i in range(T)]

for i in range(T):
    #トピックを生成
    z1 = np.dot(np.random.multinomial(1, theta[i, ], n1[i]), np.arange(topic))
    z2 = np.dot(np.random.multinomial(1, theta[i, ], n2[i]), np.arange(topic))

    #データを生成
    pt1 = np.random.poisson(np.random.gamma(12.5, 1/0.15, n1[i]))
    pt2 = np.random.poisson(np.random.gamma(12.5, 1/0.15, n2[i]))
    x1 = np.zeros((n1[i], k), dtype="int")
    x2 = np.zeros((n2[i], k), dtype="int")

    #学習データを生成
    for j in range(n1[i]):
        x1[j, ] = np.random.multinomial(pt1[j], phi[z1[j], ], 1).reshape(-1)
    x1_list[i] = np.hstack((np.repeat(1, n1[i])[:, np.newaxis], x1))
    y1_list[i] = np.dot(x1_list[i], beta[:, i]) + np.random.normal(0, Sigma[i], n1[i])

    #テストデータを生成
    for j in range(n2[i]):
        x2[j, ] = np.random.multinomial(pt2[j], phi[z2[j], ], 1).reshape(-1)
    x2_list[i] = np.hstack((np.repeat(1, n2[i])[:, np.newaxis], x2))
    y2_list[i] = np.dot(x2_list[i], beta[:, i]) + np.random.normal(0, Sigma[i], n2[i])

In [156]:
####交互方向乗数法による推定####
##データの定数を定義
#データの格納用配列
xx1_list = [i for i in range(k+1)]
xx2_list = [i for i in range(k+1)]
xy_list = [i for i in range(k+1)]

#変数のターゲット
index_target = np.zeros((k+1, k), dtype="int")
for j in range(k+1):
    index_target[j, ] = np.delete(np.arange(k+1), j)

#変数ごとに定数を定義
for i in range(k+1):
    if i%100==0:
        print(i)
    xx1 = np.zeros((T, T))
    xx2 = np.zeros((T, k))
    xy = np.repeat(0, T)
    for j in range(T):
        x = x1_list[j]; y = y1_list[j]
        xx1[j, j] = np.dot(x[:, i].T, x[:, i])
        xx2[j, ] = np.dot(x[:, i].T, x[:, index_target[i, ]])
        xy[j] = np.dot(x[:, i].T, y)

    #リストに格納
    xx1_list[i] = xx1
    xx2_list[i] = xx2
    xy_list[i] = xy

0
100
200
300
400
500
600
700
800
900
1000


In [359]:
#初期値の設定
Lambda1 = 0.1
Lambda2 = Lambda1 * np.sqrt(T)
rho = 0.25
beta[0, ] = np.array([np.mean(y1_list[j]) for j in range(T)])
beta[1:, ] = np.random.multivariate_normal(np.repeat(0, T), np.diag(np.repeat(0.25, T)), k)
gamma = beta + np.random.multivariate_normal(np.repeat(0, T), np.diag(np.repeat(0.1, T)))
u = beta + np.random.multivariate_normal(np.repeat(0, T), np.diag(np.repeat(0.1, T)))
Sigma = np.repeat(1, T)
gamma_sums1 = np.sum(np.abs(gamma))

In [360]:
beta = betat.copy()
i = 0 
xx1 = xx1_list[i]; xx2 = xx2_list[i]; xy = xy_list[i]
inv_xxv = np.linalg.inv(xx1 + np.diag(n1))
beta[i, ] = np.dot(inv_xxv, xy - np.diag(np.dot(xx2, beta[index_target[i, ]])))

In [361]:
for rp in range(100):
    #betaを更新
    for i in range(T):
        xx1 = xx1_list[i]; xx2 = xx2_list[i]; xy = xy_list[i]
        inv_xxv = np.linalg.inv(xx1 + rho*np.diag(n1))
        beta[i, ] = np.dot(inv_xxv, xy - np.diag(np.dot(xx2, beta[index_target[i, ]])) + n1*rho*(gamma[i, ] - (1/rho)*u[i, ]))

    #軟閾値作用素によりgammaを更新
    gamma = beta + u/rho
    z = np.sign(gamma)
    gamma = ((gamma - z*Lambda1) * np.array(np.abs(gamma - z*Lambda2) > Lambda2, dtype="int"))

    #uを更新
    u = u + rho*(beta - gamma)

    #平均二乗誤差和を更新
    LLt = np.repeat(0.0, T)
    for i in range(T):
        x = x1_list[i]; y = y1_list[i]
        mu = np.dot(x, gamma[:, i])
        LLt[i] = np.sum(np.power(y - mu, 2)) / n1[i]
    LL = np.sum(LLt) + Lambda1*np.sum(np.sqrt(T)*np.sqrt(np.diag(np.dot(gamma.T, gamma))))
    print(LL)

647334.2029775688
1380.406388634396
1016.8226180152919
937.0080818724576
942.5523891013247
838.7106519706533
976.6885991186966
835.412783091328
934.1069612058955
902.044759215754
932.9157858787236
835.2107763863053
1021.0327589964348
853.0708284437568
899.7261793361654
919.2190764304241
948.7790300265397
838.5532071107713
972.217197544958
847.2140410463128
954.772626591549
925.4747854159825
904.9999209793244
813.2318945932914
1029.1774379610924
897.8845654431188
896.9422852370845


KeyboardInterrupt: 

In [376]:
i = 7
print(gamma[i, ])
print(betat[i, ])

[ 0.  0.  0. -0.  0.  0. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0.  0.  0.
 -0.  0. -0.  0. -0. -0.  0.  0.  0. -0.  0. -0. -0.  0.  0.  0.  0.  0.
 -0.  0. -0. -0.  0. -0. -0. -0. -0.  0.  0.  0. -0.  0.]
[ 0.  0.  0. -0. -0. -0. -0. -0. -0.  0. -0. -0. -0.  0. -0.  0.  0. -0.
 -0.  0.  0. -0. -0.  0.  0.  0. -0.  0. -0. -0. -0. -0. -0.  0. -0.  0.
 -0.  0.  0. -0. -0.  0.  0. -0. -0. -0. -0. -0.  0.  0.]


In [377]:
gamma

array([[10.57757148, 12.19962336, 10.07324333, ..., 10.49617708,
         7.72347675,  9.95409896],
       [ 0.        , -0.        , -0.        , ...,  0.        ,
        -0.        , -0.        ],
       [ 0.        , -0.        , -0.        , ...,  0.        ,
        -0.        , -0.        ],
       ...,
       [-0.        ,  0.        , -0.        , ...,  0.        ,
         0.        , -0.        ],
       [-0.        , -0.        ,  0.        , ..., -0.        ,
        -1.80487394, -0.        ],
       [ 0.        , -0.        ,  0.        , ..., -0.        ,
        -0.        , -0.        ]])

In [258]:
gamma_sums

20574.815409111918

24462.654994735407

In [120]:
i = 1
j = 0
b = np.repeat(0.0, T)
for j in range(T):
    x = x1_list[j]; y = y1_list[j]
    x1 = x[:, i]
    x2 = x[:, index_target[i, ]]
    beta_target = beta[index_target[i,], j]
    er_y = y - np.dot(x2, beta_target)
    b[j] = np.dot(er_y, x1) / n1[j]

In [125]:
z = np.sign(b)
np.abs(b - z*Lambda2)  

array([0.96071394, 0.99241794, 0.99902468, 0.98846513, 0.97913602,
       0.99757698, 0.9987417 , 0.99878674, 0.9936527 , 0.99074908,
       0.99842057, 0.9986379 , 0.97261704, 0.99591465, 0.99563903,
       0.99628673, 0.99628649, 0.99525764, 0.99124077, 0.9921314 ,
       0.99682234, 0.99727353, 0.98901611, 0.9934912 , 0.98825055,
       0.99904567, 0.98468766, 0.98981052, 0.98451333, 0.99330006,
       0.9873775 , 0.99992027, 0.99182028, 0.99880589, 0.98791264,
       0.99099412, 0.99887385, 0.97365691, 0.99902426, 0.99419288,
       0.99247971, 0.99915752, 0.99781236, 0.98617359, 0.97270765,
       0.99277895, 0.9979184 , 0.99182532, 0.97005212, 0.99541826,
       0.99653041, 0.99893189, 0.98658035, 0.99805112, 0.99812589,
       0.98622936, 0.98651728, 0.99938825, 0.98311241, 0.99760657,
       0.9985575 , 0.99872517, 0.96632635, 0.9896115 , 0.99170338,
       0.99459624, 0.99999187, 0.9805385 , 0.98861949, 0.99415174,
       0.99898757, 0.99775301, 0.99081667, 0.99569988, 0.96394

In [66]:



z = np.sign(b)
b * np.array(np.abs(b - z*Lambda) > Lambda, dtype="int")

-0.0

In [68]:
beta_target

array([[10.20123751,  9.11631827,  8.31429717, ..., 10.86326689,
        10.53206037,  9.84469279],
       [ 0.05132926, -0.06632402,  0.4024639 , ..., -0.49786756,
         0.23132907,  0.06516642],
       [-0.65985498,  0.38733991, -0.51967357, ...,  0.02462773,
        -0.15548003,  0.38731719],
       ...,
       [-0.24424392, -0.67262604, -0.47692225, ...,  0.73802663,
        -0.07706822, -0.23223432],
       [-0.8196179 , -0.54582875,  0.73158199, ...,  0.13091347,
        -0.02622204,  0.2585272 ],
       [-0.54729017,  0.11243547, -0.48848592, ..., -0.56552892,
        -0.15626555,  0.32250439]])

-1.0